# 목표 Multi Q-table
order box 타겟(A, B, ..., Q)에 대한 q-table을 q-learning으로 수렴시키게 만든다.   
총 18개(A, B, ..., Q, [9,4] = start point)의 수렴된 table을 만들어야 한다.   
order box를 얻은 경우 다음 order box를 향한다.    
start -> A -> B -> end로 움직이는 알고리즘을 짠다.

진구님 [깃허브 P-Q러닝 참고](https://github.com/stormofdawn/path-finding-rl/blob/main/P-Q%EB%9F%AC%EB%8B%9D.ipynb)

In [ ]:
import random
from string import ascii_uppercase
# from draw_utils import * 
# from pyglet.gl import *
import numpy as np
import pandas as pd
import os

In [8]:
# reward
move_reward = -1 
obs_reward = -10     # 한 번 장애물에 부딪치면, 10번에 1번정도만 장애물쪽에 이동하도록 함
goal_reward = 1000   # Q-value의 빠른 수렴을 위해 넉넉하게 리워드를 부여
# print('reward:' , move_reward, obs_reward, goal_reward) 
inds = list(ascii_uppercase)[:17]  # 클래스 활용이 미숙해 전역변수로 변환
inds.append("T")     # T-state (최종 목적지=시작 목적지)를 추가해 줌 (box.csv에도 추가함)
Q_table_name_lst = inds
print(inds)
Q_tables = [] #Q_table_A : np.zero((10, 9, 4)) 형태의 딕셔너리 혹은 Structure Array 형태로 만들어보고 싶었으나 실패
for Q_table_name in Q_table_name_lst:
    globals()["Q_table_{}".format(Q_table_name)] = np.zeros((10, 9, 4)) 
    Q_tables.append(globals()["Q_table_{}".format(Q_table_name)]) 
#일단 완전히 초기화된 Q-Table들의 리스트인 Q-Tables를 만듬, 그냥 np.zeros((18, 10, 9, 4)) 해도 됐을듯 ㅠㅠ

local_path = "/content/drive/Othercomputers/MacBook_Air/path-finding-rl"

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'T']


In [ ]:
box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))

In [ ]:
box_coor_index =[]
for i in range(len(box_data)):
    box_coor_index.append([box_data['row'][i], box_data['col'][i]])
print(box_coor_index)

[[5, 0], [4, 0], [3, 0], [2, 0], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [2, 8], [3, 8], [4, 8], [5, 8]]


### Q-Tables의 인덱스 = box_coor_index의 인덱스

In [ ]:
class Simulator_exploration:  #단순 탐색을 위한 환경
    def __init__(self):
        '''
        height : 그리드 높이
        width : 그리드 너비 
        inds : A ~ Q alphabet list
        '''
        # Load train data
        self.files = pd.read_csv(os.path.join(local_path, "./data/Q_Finder.csv")) #각각 A부터 T까지 하나를 목표로하는 csv파일을 만듬
        self.height = 10 #세로 10
        self.width = 9 # 가로 9  #A~Q까지 알파벳(target) 선언
        self.failcount = 0
        self.successcount = 0
    def set_box(self):
        '''
        아이템들이 있을 위치를 미리 정해놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))

        # 물건이 들어있을 수 있는 경우
        for box in box_data.itertuples(index = True, name ='Pandas'):  #판다스 데이터를 튜플로 iter해준 것 같다. 행/열 위치정보로 각 알파벳이 표시됨
            self.grid[getattr(box, "row")][getattr(box, "col")] = 100   #아이템이 없는 경우 100

        # 물건이 실제 들어있는 경우
        order_item = list(set(inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]
        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = -100 # 아이템이 있는 경우 -100 <- 값이 지정되면 리턴으로 사용가능?
            # local target에 가야 할 위치 좌표 넣기
            self.local_target.append(
                [getattr(order_box, "row"),
                 getattr(order_box, "col")]
                )# 타겟 위치 지정 완료
            # print(order_box)
            
        # print(self.local_target)
        # self.local_target.append([9,4])  # 마지막 위치를 추가해주지 않음

        # 알파벳을 Grid에 넣어서 -> grid에 2Dconv 적용 가능

    def set_obstacle(self):
        '''
        장애물이 있어야하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = 0 #장애물 행,열 지정

    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.

        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기셋팅 된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items : 이번 에피소드에서 가져와야하는 아이템들
        terminal_location : 현재 에이전트가 찾아가야하는 목적지
        local_target : 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc : 현재 위치
        '''

        # initial episode parameter setting
        self.epi = epi
        self.items = list(self.files.iloc[self.epi])[0]
        self.cumulative_reward = 0
        self.terminal_location = None
        self.local_target = []
        self.actions = [] # 정책에 따라 시간 순서대로 action 리스트에 append해주면 될듯

        # initial grid setting
        self.grid = np.ones((self.height, self.width), dtype="float16") #초기는 전부0

        # set information about the gridworld
        self.set_box() #빈박스 + 타겟 박스
        self.set_obstacle() #장애물

        # start point를 grid에 표시
        self.curloc = [np.random.randint(10), np.random.randint(9)] #시작 위치 초기화 (랜덤 위치)
        # obstacle, target 등의 위치를 빼고 시작한다면 더 빠른 수렴이 가능하겠지만, 어차피 그 위치의 q-value는 사용될리 없으므로 완전 랜덤으로 해줌
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = -5       
        
        self.done = False
        
        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1

        return int(new_x), int(new_y)


    def get_reward(self, new_x, new_y, out_of_boundary):
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.

        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward
                       
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[new_x][new_y] == 0:
                reward = obs_reward  

            # 현재 목표에 도달한 경우
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                reward = goal_reward

            # 그냥 움직이는 경우 
            else:
                reward = move_reward

        return reward

    def step(self, action):
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.

        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터

        :rtype: numpy.ndarray, float, float, bool, bool/str

        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''
        # print(self.local_target)
        self.terminal_location = self.local_target[0]
        cur_x,cur_y = self.curloc
        self.actions.append((cur_x, cur_y))

        goal_ob_reward = False
        
        new_x, new_y = self.apply_action(action, cur_x, cur_y)

        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]

        # 바깥으로 나가는 경우 종료
        if any(out_of_boundary):
            self.done = True
            goal_ob_reward = True
            self.failcount += 1 # 실패 카운터
        else:
            # 장애물에 부딪히는 경우 종료
            if self.grid[new_x][new_y] == 0:
                self.done = True
                goal_ob_reward = True
                self.failcount += 1 # 실패 카운터

            # 현재 목표에 도달한 경우, 다음 목표설정
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                self.successcount +=1 # 성공 카운터

                # end point 일 때
                # if [new_x, new_y] == [9,4]:
                self.done = True # 목표에 도착하면 바로 done 수행

                # self.local_target.remove(self.local_target[0]) 
                # self.grid[cur_x][cur_y] = 1
                # self.grid[new_x][new_y] = -5
                goal_ob_reward = True
                self.curloc = [new_x, new_y]
            else:
                # 그냥 움직이는 경우 
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                self.curloc = [new_x,new_y]
                
        reward = self.get_reward(new_x, new_y, out_of_boundary)
        self.cumulative_reward += reward

        # if self.done == True:
        #     if [new_x, new_y] == [9, 4]: #초기위치로 도달했을 때,
        #         if self.terminal_location == [9, 4]: # 목표와 같을때만 gif로 만들어줌
        #             # 완료되면 GIFS 저장
        #             goal_ob_reward = 'finish'
        #             height = 10
        #             width = 9
        #             display = Display(visible=False, size=(width, height))
        #             display.start()
        #
        #             start_point = (9, 4)
        #             unit = 50
        #             screen_height = height * unit
        #             screen_width = width * unit
        #             log_path = "./logs"
        #             data_path = "./data"
        #             render_cls = Render(screen_width, screen_height, unit, start_point, data_path, log_path)
        #             for idx, new_pos in enumerate(self.actions):
        #                 render_cls.update_movement(new_pos, idx+1)
        #
        #             render_cls.save_gif(self.epi)
        #             render_cls.viewer.close()
        #             display.stop()
        # 코랩환경이라 그림그리기는 주석처리함
        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward

In [ ]:
class QAgent_exploration():
    def __init__(self):
        self.eps = 0.0
        self.eps_decay = 0.00
    # 랜덤한 위치 -> 굳이 입실론을 통해 탐색하지 않아도 모든 지점에서 시작할 확률이 같으므로 입실론은 무의미
    
    def select_action(self, curloc, index):
        # Q-table은 단순 그리디로 이동, 목표에 따라 Q-Tables에서 인덱스를 알아야하므로 인덱스를 인수로 추가해줌
        x, y = curloc
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = Q_tables[index][x,y,:] # 목표의 인덱스 = 목표의 Q-Table
            action = np.argmax(action_val)
        return action

    def update_table(self, transition, index):
        s, a, r , s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime, index) 
        Q_tables[index][x,y,a] = Q_tables[index][x,y,a] + 0.1*(r + np.max(Q_tables[index][next_x, next_y, :]) - Q_tables[index][x,y,a])
        #Q러닝 업데이트 식을 이용, 마찬가지로 index를 인수로 추가해 table에 접근함

    def anneal_eps(self):
        self.eps -= self.eps_decay
        self.eps = max(self.eps, 0.1) # 만에하나 한번도 못가본 장소가 있을까봐 0.1만 뒀음 (0이라도 상관없을 듯)

In [ ]:
def main():
    env = Simulator_exploration()
    agent = QAgent_exploration()
    episode = 10000
    files = pd.read_csv(os.path.join(local_path, "./data/Q_Finder.csv"))

    for n_epi in range(episode): #에피소드 변수만큼 실행

        for targetidx in range(len(files)): # 목표를 A~T까지 돌아가면서 수행 + 업데이트
            items = list(files.iloc[targetidx])[0]
            done = False
            if n_epi % 20 == 0:
                print(f'에피소드 :{n_epi} ')
            env.reset(targetidx)
            while not done: # 한 에피소드가 끝날때 까지
                s = env.curloc
                a = agent.select_action(s, targetidx)
                _, r, cum_reward, done, goal_ob_reward = env.step(a)
                s_prime = env.curloc
                agent.update_table((s, a, r, s_prime), targetidx)
                s = s_prime
            agent.anneal_eps()

In [ ]:
main()

In [ ]:
import pickle
file = open("/content/drive/MyDrive/git_path_finding/path-finding-rl/data/save.txt", "wb") 
pickle.dump(Q_tables, file)
file.close

### 텍스트로 Q테이블 저장해보려고 했는데 안됨... 알려주세요

In [ ]:
with open('Q_tables(20000).pickle', 'wb') as f:
    pickle.dump(Q_tables, f)

### 피클로 저장할 수 있다고 해서 저장 -> data 폴더에 넣어둠
경로 지정은 어떻게 할지 모르겠습니다.

cd 해서 폴더 위치로 오면 될지도?

In [ ]:
with open('Q_tables(20000).pickle','rb') as f:    #'rb'라는 점을 주의하라
    Q_tables = pickle.load(f)

위치 찾아서 위 테이블을 로드하면 어디서나 사용가능

In [ ]:
print(Q_tables)

### Env

In [ ]:
class Simulator:
    def __init__(self):
        '''
        height : 그리드 높이
        width : 그리드 너비 
        inds : A ~ Q alphabet list
        '''
        # Load train data
        self.files = pd.read_csv(os.path.join(local_path, "./data/factory_order_test.csv"))
        self.height = 10 #세로 10
        self.width = 9 # 가로 9  #A~Q까지 알파벳(target) 선언
        self.failcount = 0
        self.successcount = 0
    def set_box(self):
        '''
        아이템들이 있을 위치를 미리 정해놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))

        # 물건이 들어있을 수 있는 경우
        for box in box_data.itertuples(index = True, name ='Pandas'):  #판다스 데이터를 튜플로 iter해준 것 같다. 행/열 위치정보로 각 알파벳이 표시됨
            self.grid[getattr(box, "row")][getattr(box, "col")] = 100   #아이템이 없는 경우 100

        # 물건이 실제 들어있는 경우
        order_item = list(set(inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]
        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = -100 # 아이템이 있는 경우 -100 <- 값이 지정되면 리턴으로 사용가능?
            # local target에 가야 할 위치 좌표 넣기
            self.local_target.append(
                [getattr(order_box, "row"),
                 getattr(order_box, "col")]
                )# 타겟 위치 지정 완료
            # print(order_box)
            
        # print(self.local_target)
        self.local_target.append([9,4])  # 마지막 타겟 = 출발위치

        # 알파벳을 Grid에 넣어서 -> grid에 2Dconv 적용 가능

    def set_obstacle(self):
        '''
        장애물이 있어야하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = 0 #장애물 행,열 지정

    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.

        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기셋팅 된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items : 이번 에피소드에서 가져와야하는 아이템들
        terminal_location : 현재 에이전트가 찾아가야하는 목적지
        local_target : 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc : 현재 위치
        '''

        # initial episode parameter setting
        self.epi = epi
        self.items = list(self.files.iloc[self.epi])[0]
        self.cumulative_reward = 0
        self.terminal_location = None
        self.local_target = []
        self.actions = [] # 정책에 따라 시간 순서대로 action 리스트에 append해주면 될듯

        # initial grid setting
        self.grid = np.ones((self.height, self.width), dtype="float16") #초기는 전부0

        # set information about the gridworld
        self.set_box() #빈박스 + 타겟 박스
        self.set_obstacle() #장애물

        # start point를 grid에 표시
        self.curloc = [9, 4] #시작 위치 초기화 (인덱스 0부터 시작이므로 10번째줄, 5번째 열임)
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = -5       
        
        self.done = False
        
        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1

        return int(new_x), int(new_y)


    def get_reward(self, new_x, new_y, out_of_boundary):
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.

        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward
                       
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[new_x][new_y] == 0:
                reward = obs_reward  

            # 현재 목표에 도달한 경우
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                reward = goal_reward

            # 그냥 움직이는 경우 
            else:
                reward = move_reward

        return reward

    def step(self, action):
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.

        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터

        :rtype: numpy.ndarray, float, float, bool, bool/str

        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''
        # print(self.local_target)
        self.terminal_location = self.local_target[0]
        cur_x,cur_y = self.curloc
        self.actions.append((cur_x, cur_y))

        goal_ob_reward = False
        
        new_x, new_y = self.apply_action(action, cur_x, cur_y)

        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]

        # 바깥으로 나가는 경우 종료
        if any(out_of_boundary):
            self.done = True
            goal_ob_reward = True
            self.failcount += 1
        else:
            # 장애물에 부딪히는 경우 종료
            if self.grid[new_x][new_y] == 0:
                self.done = True
                goal_ob_reward = True
                self.failcount += 1

            # 현재 목표에 도달한 경우, 다음 목표설정
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                

                # end point 일 때
                if [new_x, new_y] == [9,4]:
                    self.done = True # 마지막 목표가 설정되면 done을 True로 설정
                    self.successcount +=1

                self.local_target.remove(self.local_target[0]) # 목표 달성시 목표 맨 앞 리스트를 지워준다.
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                goal_ob_reward = True
                self.curloc = [new_x, new_y]
            else:
                # 그냥 움직이는 경우 
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                self.curloc = [new_x,new_y]
                
        reward = self.get_reward(new_x, new_y, out_of_boundary)
        self.cumulative_reward += reward

        # if self.done == True:
        #     if [new_x, new_y] == [9, 4]: #초기위치로 도달했을 때,
        #         if self.terminal_location == [9, 4]: # 목표와 같을때만 gif로 만들어줌
        #             # 완료되면 GIFS 저장
        #             goal_ob_reward = 'finish'
        #             height = 10
        #             width = 9
        #             display = Display(visible=False, size=(width, height))
        #             display.start()
        #
        #             start_point = (9, 4)
        #             unit = 50
        #             screen_height = height * unit
        #             screen_width = width * unit
        #             log_path = "./logs"
        #             data_path = "./data"
        #             render_cls = Render(screen_width, screen_height, unit, start_point, data_path, log_path)
        #             for idx, new_pos in enumerate(self.actions):
        #                 render_cls.update_movement(new_pos, idx+1)
        #
        #             render_cls.save_gif(self.epi)
        #             render_cls.viewer.close()
        #             display.stop()
        
        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward

In [ ]:
class QAgent_exploitation(): #테스트 클래스, 마찬가지로 입실론은 안써도 된다.
    def __init__(self):
        self.eps = 0.0  # exploitation만 하면됨
        self.eps_decay = 0.00

    
    def select_action(self, curloc, index):
        # index인수로 받아서 접근만 추가함
        x, y = curloc
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = Q_tables[index][x,y,:]
            action = np.argmax(action_val)
        return action

    def update_table(self, transition, index):
        s, a, r , s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime, index) 
        # Q_tables[index][x,y,a] = Q_tables[index][x,y,a] + 0.1*(r + np.max(Q_tables[index][next_x, next_y, :]) - Q_tables[index][x,y,a])
        # table은 더이상 업데이트 안해도됨, Q-Table만 따라 이동하면 된다.


    def anneal_eps(self):
        self.eps -= self.eps_decay
        self.eps = max(self.eps, 0.0)

In [ ]:
def main():
    env = Simulator()
    agent = QAgent_exploitation()
    episode = len(env.files)

    for n_epi in range(episode):
        alst = []
        acount = 0
        done = False
        env.reset(n_epi)
        while not done: # 에피소드가 끝날때 까지 반복
            targetidx = box_coor_index.index(env.local_target[0])
            s = env.curloc
            a = agent.select_action(s, targetidx)
            alst.append(a) #액션들을 alst에 모아둠 -> 나중에 시각화하려구
            acount += 1 #액션 수
            _, r, cum_reward, done, goal_ob_reward = env.step(a)
            s_prime = env.curloc
            agent.update_table((s, a, r, s_prime), targetidx)
            s = s_prime
        agent.anneal_eps()
        print(f'에피소드 :{n_epi}, 액션 수: {acount}, 리턴: {cum_reward}, 성공율 : {(env.successcount/(env.successcount+env.failcount+0.00001)*100):2f} %')
        if n_epi%20 ==0:
            print(alst) #20회마다 액션리스트만 print

In [ ]:
main()